In [11]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [1]:
# Configurations spiros

import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_remote_path_root = pathlib.Path('/webdav')
conf_remote_path_ahn = pathlib.Path('/webdav/ahn')
conf_remote_path_retiled = pathlib.Path('/webdav/retiled/')
conf_remote_path_targets = pathlib.Path('/webdav/targets')
conf_local_tmp = pathlib.Path('/tmp')

param_hostname = 'https://lifewatch.lab.uvalight.net:32443'
param_login = 'ZDFmNDg0Nzk0ZTJi'
param_password = 'NDBjMmMzODJkYTBh'


conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}

In [ ]:
def send_annotation(start=None,end=None,message=None,tags=None):
    if not tags:
        tags = []
    
    tags.append(theNotebook)
    
    headers = {
        'Accept':'application/json',
        'Content-Type': 'application/json',
    }
    
    data ={
      "dashboardId":1,
    #   "panelId":8,
      "time":start,
      "timeEnd":end,
      "created": end,
      "tags":tags,
      "text": message
    }
    resp = requests.post(grafana_base_url+'/api/annotations',verify=False,auth=('admin', grafana_pwd),headers=headers,json=data)
    
    data ={
      "dashboardId":2,
    #   "panelId":8,
      "time":start,
      "timeEnd":end,
      "created": end,
      "tags":tags,
      "text": message
    }
    resp = requests.post(grafana_base_url+'/api/annotations',verify=False,auth=('admin', grafana_pwd),headers=headers,json=data)

## Fetching Laz Files from remote WebDAV

In [6]:
# Fetch Laz Files spi
import pathlib
laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_ahn.as_posix())
             if f.lower().endswith('.laz')]

## Retiling of big files into smaller tiles

In [7]:
# Retiling spi

remote_path_retiled = conf_remote_path_retiled

grid_retile = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}


retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_ahn.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

file = laz_files
retiler = Retiler(file).config(retiling_input).setup_webdav_client(conf_wd_opts)
retiler.run()

2022-01-31 11:05:12,133 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-01-31 11:05:12,135 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-01-31 11:05:12,137 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/ahn/C_18HZ2.LAZ ...
2022-01-31 11:05:13,265 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-01-31 11:05:13,267 -                        laserfarm.retiler -       INFO - Setting up the target grid
2022-01-31 11:05:13,270 -                        laserfarm.retiler -       INFO - Splitting file /tmp/pipeline_input/C_18HZ2.LAZ with PDAL ...
2022-01-31 11:05:14,418 -                        laserfarm.retiler -       INFO - ... splitting completed.
2022-01-31 11:05:14,421 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2022-01-31 11:05:14,424 -                        l

## Fetching retilied files (tiles) from remote WebDAV

In [6]:
# Fetch Tiles spi

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

## Extract features from tiles

In [11]:
# Feature Extraction spi

t = tiles[2]

tile_mesh_size = 10.
features = ['perc_95_normalized_height']

grid_feature = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

feature_extraction_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_retiled.as_posix(),
    'load': {'attributes': ['raw_classification']},
    'normalize': 1,
    'apply_filter': {
        'filter_type': 'select_equal', 
        'attribute': 'raw_classification',
        'value': [1, 6]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        **grid_feature
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'pushremote': conf_remote_path_targets.as_posix(),
#     'cleanlocalfs': {}
}
idx = (t.split('_')[1:])
processing = DataProcessing(t, tile_index=idx).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
processing.run()

2022-01-10 11:39:25,502 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-01-10 11:39:25,503 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-01-10 11:39:25,505 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/retiled/tile_278_391 ...
2022-01-10 11:39:28,154 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-01-10 11:39:28,159 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2022-01-10 11:39:28,161 -                laserfarm.data_processing -       INFO - ... loading /tmp/pipeline_input/tile_278_391/C_01GN2_1.LAZ
2022-01-10 11:39:28,175 -                          pylas.lasreader -      ERROR - lazrs failed to decompress points: lazrs is not installed
2022-01-10 11:39:38,522 -                laserfarm.data_processing -       INFO - ... loading completed.
2022-01-10 11:39:38,523

FileExistsError: Path /tmp/pipeline_output/perc_95_normalized_height/tile_278_391.ply already exists!

2022-01-10 11:51:44,967 -                                     root -       INFO - ['C_18HZ2.LAZ', 'C_19HZ2.LAZ', 'C_01GN2.LAZ', 'C_50GZ2.LAZ']
2022-01-10 11:51:52,511 -                                     root -       INFO - ['C_18HZ2.LAZ', 'C_19HZ2.LAZ', 'C_01GN2.LAZ', 'C_50GZ2.LAZ']
2022-01-10 11:51:58,172 -                                     root -       INFO - ['C_18HZ2.LAZ', 'C_19HZ2.LAZ', 'C_01GN2.LAZ', 'C_50GZ2.LAZ']
2022-01-10 11:52:24,048 -                                     root -       INFO - ['C_18HZ2.LAZ', 'C_19HZ2.LAZ', 'C_01GN2.LAZ', 'C_50GZ2.LAZ']


In [ ]:
# GeoTIFF Export sp

feature = features
remote_path_geotiffs = conf_remote_path_ahn.parent / 'geotiffs'

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=feature, bands=feature).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()